In [1]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import nibabel as nib
import random
from scipy import stats
from scipy import fft
import random
import pywt.data
from PIL import Image
import pandas as pd
import seaborn as sns
import os
import pickle

In [12]:
# assign directory
import os
BRANDON = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
YASH = '/Users/yashd/Desktop/hierarchical-bayesian-model-validation/'
HANNAH = '/Users/hannahchung/Documents/GitHub/hierarchical-bayesian-model-validation/'

ROOT_DIR = HANNAH
data_dir = '/Users/hannahchung/Downloads/cleaned/'
# iterate over files in that directory
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)

In [13]:
os.chdir(ROOT_DIR + "dataset-preparation/")

In [14]:
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]


In [19]:
data = np.load(file_list[0])
print(data.files)
image = data['arr_0']
np.nansum(image)

['arr_0']


np.float32(203686670.0)

In [20]:
def convert_to_wavelet_basis_3d(folder_dir, basis="db1", normalized = False):
    file_list = [os.path.join(folder_dir, filename) for filename in os.listdir(folder_dir) if filename.endswith('.npz')]
    #Setup df Dict
    data = np.load(file_list[0])
    image = data['arr_0']
    first_image = pywt.wavedecn(image, basis)
    layer_len = len(first_image)
    direction_names = first_image[1].keys()
    direction_num = len(direction_names)
    print(str(layer_len) + " layers being used")
    

    #Fill DF DICT
    layer_arr = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    orientation = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    data_arr = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    cnt = 0
    for k in range(len(file_list)):

        data = np.load(file_list[k])
        image = data['arr_0']


        if normalized:
            std= np.std(image)
            mean = np.mean(image)
            image = (image- mean)/std

    
        transformed = pywt.wavedecn(image, 'db1')
        

        arr = np.array(transformed[0][0]).flatten()
        layer_arr[cnt] = 1
        orientation[cnt] =  "L1"
        data_arr[cnt] = arr.flatten()
        cnt += 1

        for i in range(1, layer_len): 
            for j in direction_names:
                
                arr = np.array(transformed[i][j]).flatten()
                layer_arr[cnt] = i+1
                orientation[cnt] =  j
                data_arr[cnt] = arr.flatten()
                cnt += 1

    df = pd.DataFrame()

    df["layer"] = layer_arr
    df["orientation"] = orientation
    df["data"] = data_arr
    new_df = pd.DataFrame(columns=["layer", "orientation", "data"])
    for lo, sf in df.groupby(["layer", "orientation"])[["data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
        new_df.loc[len(new_df.index)] = [lo[0], lo[1],  np.hstack(sf['data'])]
    
    return new_df

In [22]:
df = convert_to_wavelet_basis_3d(data_dir)

8 layers being used


In [28]:
df

,layer,orientation,data
0,1,L1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,2,aad,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,2,ada,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,2,add,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,2,daa,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
5,2,dad,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
6,2,dda,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
7,2,ddd,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
8,3,aad,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
9,3,ada,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [43]:
arr = df[(df["layer"] == 7) & (df["orientation"] == "daa")].iloc[0]["data"]
print(f"Total length: {len(arr)}")
print(f"NaN count: {np.isnan(arr).sum()}")
print(f"Valid (non-NaN) count: {np.isfinite(arr).sum()}")
print("Sample non-NaN values:", arr[np.isfinite(arr)][:10])


Total length: 89600000
NaN count: 47037115
Valid (non-NaN) count: 42562885
Sample non-NaN values: [-234.1743   -278.30728  -315.79355  -369.09445  -371.2901   -324.2051
 -224.17853  -108.54943   -45.135532   36.623333]
